In [2]:
# Database management
import sqlalchemy as sa
import psycopg2

# Data wrangling
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta



# Data visualisation
import matplotlib.pylab as plt
%matplotlib inline
import seaborn
seaborn.set()

# utilities
import pprint as pp

In [3]:
# load up database scripts and connect to db
conn_eops = {'user': 'uds_read',
            'host': 'UCLVLDDDTAEPS02',
            'port': 5432,
            'password': 'klq;wed-0i439^^%',
            'database': 'uds'      
            }

def gen_connection_string(d, engine='postgresql'):
    """Generates a SQLAlchemy connection string from a dictionary"""
    # e.g. # engine = create_engine('postgresql+psycopg2://steve@localhost:5432/omop')
    # TODO @later permit other interfaces
    d['port'] = str(d['port'])

    if engine == 'postgresql':
        if d.get('password'):
            return 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(**d)
        else:
            return 'postgresql+psycopg2://{user}@{host}:{port}/{database}'.format(**d)

    elif engine == 'psycopg2':
        if d.get('password'):
            return 'host={host} dbname={database} user={user} password={password} port={port}'.format(**d)
        else:
            return 'host={host} dbname={database} user={user} port={port}'.format(**d)

In [4]:
URL = gen_connection_string(conn_eops, engine='postgresql')
DSN = gen_connection_string(conn_eops, engine='psycopg2')
conn = psycopg2.connect(DSN, options=f'-c search_path=live')

In [5]:
engine = sa.create_engine(URL, connect_args= {'options': '-c search_path=live'})
inspector = sa.inspect(engine)
print(inspector.get_table_names())

['ids_progress', 'tbl_ids_master', 'person', 'person_mrn', 'ids_effect_logging', 'mrn_encounter', 'patient_fact', 'encounter', 'patient_property', 'attribute', 'mrn']


In [6]:
metadata = sa.MetaData()
metadata.reflect(bind=engine)
ttables = metadata.sorted_tables
ttables = {t.name:t.columns.keys() for t in ttables}
ttables.keys()

dict_keys(['attribute', 'encounter', 'ids_effect_logging', 'ids_progress', 'mrn', 'person', 'tbl_ids_master', 'mrn_encounter', 'patient_fact', 'person_mrn', 'patient_property'])

In [8]:
# let's inspect the columns
pd.read_sql('attribute', engine)

,attribute_id,added_time,description,result_type,short_name
0,1,2019-03-04 17:00:00+00:00,First name,1,F_NAME
1,2,2019-03-04 17:00:00+00:00,Middles names separated by spaces,1,M_NAME
2,3,2019-03-04 17:00:00+00:00,Family name,1,L_NAME
3,4,2019-03-04 17:00:00+00:00,Fact group containing name data,0,NAMING
4,5,2019-03-05 11:33:00+00:00,Fact group about a hospital visit,0,HOSP_VISIT
5,6,2019-03-19 16:40:00+00:00,Fact group about a bed visit,0,BED_VISIT
6,7,2019-03-05 11:33:00+00:00,Time of arrival,2,ARRIVAL_TIME
7,8,2019-03-05 11:33:00+00:00,Time of departure,2,DISCH_TIME
8,9,2019-03-19 16:40:00+00:00,Location,1,LOCATION
9,10,2019-03-20 11:56:00+00:00,Parent visit,3,VISIT_PAREN


In [18]:
pds = pd.read_sql(
    """select distinct on (pp1.value_as_string) 
        pp1.value_as_string, 
        pp2.value_as_string,
        pp1.parent_fact 
     from patient_property pp1 
     left join patient_property pp2 
        on pp1.parent_fact = pp2.parent_fact 
        where pp2.attribute=24 
        or
        pp1.attribute = 23;
    """
    , engine)
pds

,value_as_string,value_as_string,parent_fact
0,1433,F,98973442
1,1AD,F,53806148
2,1CAM,F,96397747
3,1EG,F,63468074
4,1HD,F,51929671
5,1HE,F,50925959
6,1LY,F,96191691
7,1RE,F,77823105
8,1RHH,F,89102354
9,1RHT,F,31368473


In [22]:
res = pd.concat([pds, pds.value_as_string.str.split('\\^', expand=True)], axis=1)
res

,value_as_string,0,1,2
0,GWNC^CHALF WGC SR21^SR21-21,GWNC,CHALF WGC SR21,SR21-21
1,RBNQ^Q3NRU WAITING^WAIT,RBNQ,Q3NRU WAITING,WAIT
2,T12S^T12S BY04^BY04-27,T12S,T12S BY04,BY04-27
3,T01^T01 BY04^BY04-53B,T01,T01 BY04,BY04-53B
4,MVNQ^Q6MV BY03^BY03-06,MVNQ,Q6MV BY03,BY03-06
5,CSW1^CLVST DAY BY02^BY02-01,CSW1,CLVST DAY BY02,BY02-01
6,T07S^T07S BY13^BY13-57,T07S,T07S BY13,BY13-57
7,SPEV^OFF EVERGRN BY06^BY06-14,SPEV,OFF EVERGRN BY06,BY06-14
8,T07^T07N BY03^BY03-18,T07,T07N BY03,BY03-18
9,SPEV^OFF EVERGRN BY02^BY02-05,SPEV,OFF EVERGRN BY02,BY02-05


In [23]:
res.sort_values(by=0).to_csv('extract_locations.csv')